<a href="https://colab.research.google.com/github/advayramesh/stockAnalyzer/blob/master/quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance langchain_pinecone openai python-dotenv langchain-community sentence_transformers groq

In [2]:
import os
import json
import requests
import numpy as np
import concurrent.futures
import time
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from dataclasses import dataclass
from typing import List, Dict, Any
import yfinance as yf
from groq import Groq
from google.colab import userdata
from langchain.schema import Document  # Import for Document

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
@dataclass
class RetrievedContext:
    content: str
    similarity_score: float
    metadata: Dict[Any, Any]

In [4]:
class EnhancedRAGModel:
    def __init__(self, embedding_model_name: str = "sentence-transformers/all-mpnet-base-v2", similarity_threshold: float = 0.6, max_contexts: int = 8):
        self.embedding_model = SentenceTransformer(embedding_model_name)
        self.similarity_threshold = similarity_threshold
        self.max_contexts = max_contexts
        self.hf_embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

    def retrieve_and_rank(self, query: str, vector_store: PineconeVectorStore, filters: Dict = None) -> List[RetrievedContext]:
        query_embedding = self.embedding_model.encode(query)
        raw_matches = vector_store.similarity_search_with_score(query, k=self.max_contexts * 2)

        ranked_contexts = []
        for doc, score in raw_matches:
            doc_embedding = self.embedding_model.encode(doc.page_content)
            semantic_similarity = cosine_similarity(
                query_embedding.reshape(1, -1),
                doc_embedding.reshape(1, -1)
            )[0][0]

            if filters and not self._matches_filters(doc.metadata, filters):
                continue

            if semantic_similarity >= self.similarity_threshold:
                ranked_contexts.append(
                    RetrievedContext(
                        content=doc.page_content,
                        similarity_score=semantic_similarity,
                        metadata=doc.metadata
                    )
                )

        ranked_contexts.sort(key=lambda x: x.similarity_score, reverse=True)
        return ranked_contexts[:self.max_contexts]

    def _matches_filters(self, metadata: Dict, filters: Dict) -> bool:
        for metric, value in filters.items():
            if metric == 'marketCap' and metadata.get('Market Cap', 0) < value:
                return False
            elif metric == 'sector' and metadata.get('Sector', '').lower() != value.lower():
                return False
        return True

    def generate_augmented_prompt(self, query: str, contexts: List[RetrievedContext]) -> str:
        context_texts = []
        for ctx in contexts:
            metrics = {
                'Ticker': ctx.metadata.get('Ticker', 'N/A'),
                'Name': ctx.metadata.get('Name', 'N/A'),
                'Sector': ctx.metadata.get('Sector', 'N/A'),
                'Industry': ctx.metadata.get('Industry', 'N/A'),
                'Market Cap': ctx.metadata.get('Market Cap', 'N/A'),
                'Revenue': ctx.metadata.get('Revenue', 'N/A'),
                'P/E Ratio': ctx.metadata.get('P/E Ratio', 'N/A')
            }

            metrics_str = "\n".join(f"{k}: {v}" for k, v in metrics.items())

            context_text = f"""
            --- Stock Information ---
            {metrics_str}

            Business Description:
            {ctx.content}

            Relevance Score: {ctx.similarity_score:.2f}
            ----------------------
            """
            context_texts.append(context_text)

        prompt_template = """
        Context about relevant stocks:
        {contexts}

        Query: {query}

        Please provide a detailed analysis of the stocks that match this query. For each relevant company:
        1. Explain how it relates to the query
        2. Highlight key business metrics
        3. Note any competitive advantages
        4. Consider industry position and market trends

        Focus on factual information from the context and avoid speculation.
        """

        return prompt_template.format(
            contexts="\n\n".join(context_texts),
            query=query
        )

    def get_llm_response(self, client: Groq, prompt: str, model: str = "llama-3.1-70b-versatile", temperature: float = 0.7) -> str:
        try:
            chat_completion = client.chat.completions.create(
                model=model,
                messages=[{"role": "system", "content": "You are a stock market expert."},
                          {"role": "user", "content": prompt}]
            )
            return chat_completion.choices[0].message.content
        except Exception as e:
            raise Exception(f"Error getting LLM response: {str(e)}")

    def process_query(self, query: str, vector_store: PineconeVectorStore, client: Groq, filters: Dict = None) -> str:
        ranked_contexts = self.retrieve_and_rank(query, vector_store, filters)
        augmented_prompt = self.generate_augmented_prompt(query, ranked_contexts)
        return self.get_llm_response(client, augmented_prompt)



In [5]:
# Function to fetch stock data
def get_stock_info(symbol: str) -> dict:
    data = yf.Ticker(symbol)
    stock_info = data.info
    return {
        "Ticker": stock_info.get('symbol', 'N/A'),
        'Name': stock_info.get('longName', 'N/A'),
        'Business Summary': stock_info.get('longBusinessSummary'),
        'Market Cap': stock_info.get('marketCap', 'N/A'),
        'Sector': stock_info.get('sector', 'N/A'),
        'Industry': stock_info.get('industry', 'N/A'),
        'P/E Ratio': stock_info.get('forwardPE', 'Information not available'),
        'Revenue': stock_info.get('totalRevenue', 'Information not available'),
        'Profit Margin': stock_info.get('profitMargins', 'Information not available'),
        'Operating Margin': stock_info.get('operatingMargins', 'Information not available'),
        'Book Value': stock_info.get('bookValue', 'Information not available'),
        'Price to Book': stock_info.get('priceToBook', 'Information not available'),
        'Earnings Growth': stock_info.get('earningsGrowth', 'Information not available'),
        'Dividend Rate': stock_info.get('dividendRate', 'Information not available'),
        'Dividend Yield': stock_info.get('dividendYield', 'Information not available'),
        'Payout Ratio': stock_info.get('payoutRatio', 'Information not available'),
    }

# Function to get company tickers from GitHub
def get_company_tickers():
    url = "https://raw.githubusercontent.com/advayramesh/QuantData/main/company_tickers.json"
    response = requests.get(url)
    if response.status_code == 200:
        company_tickers = json.loads(response.content.decode('utf-8'))
        return company_tickers
    else:
        raise Exception("Failed to download company tickers.")


In [7]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key
index_name = "quant"
namespace = "stock-descriptions"
hf_embeddings = HuggingFaceEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

<ipython-input-7-59d718f72484>:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()


In [8]:
# Initialize tracking lists for success and failure
successful_tickers = []
unsuccessful_tickers = []

# Load existing successful/unsuccessful tickers from text files
try:
    with open('successful_tickers.txt', 'r') as f:
        successful_tickers = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    print("No existing successful tickers file found")

try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    print("No existing unsuccessful tickers file found")

# Get the tickers to process
company_tickers = get_company_tickers()
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

No existing successful tickers file found
No existing unsuccessful tickers file found


In [9]:
from google.colab import userdata
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "quant"
namespace = "stock-descriptions"

hf_embeddings = HuggingFaceEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

<ipython-input-9-eeb847ab6650>:8: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()


In [10]:
# Initialize tracking lists
successful_tickers = []
unsuccessful_tickers = []

# Load existing successful/unsuccessful tickers
try:
    with open('successful_tickers.txt', 'r') as f:
        successful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(successful_tickers)} successful tickers")
except FileNotFoundError:
    print("No existing successful tickers file found")

try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(unsuccessful_tickers)} unsuccessful tickers")
except FileNotFoundError:
    print("No existing unsuccessful tickers file found")

No existing successful tickers file found
No existing unsuccessful tickers file found


In [11]:
import time

In [12]:
# Get last successful ticker and prepare remaining tickers
last_successful = None
if successful_tickers:
    last_successful = successful_tickers[-1]
    print(f"Last successful ticker was: {last_successful}")

# Prepare remaining tickers
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

if last_successful:
    try:
        start_index = tickers_to_process.index(last_successful) + 1
        tickers_to_process = tickers_to_process[start_index:]
        print(f"Resuming from index {start_index}, {len(tickers_to_process)} tickers remaining")
    except ValueError:
        print("Last successful ticker not found in current list")

def process_stock(stock_ticker: str) -> str:
    # Skip if already processed
    if stock_ticker in successful_tickers:
        return f"Already processed {stock_ticker}"

    try:
        # Get and store stock data
        stock_data = get_stock_info(stock_ticker)

        # Skip if no business summary
        if 'Business Summary' not in stock_data or stock_data['Business Summary'] is None:
            unsuccessful_tickers.append(stock_ticker)
            return f"Skipped {stock_ticker} - No business summary"

        stock_description = stock_data['Business Summary']
        document = Document(page_content=stock_description, metadata=stock_data)
        return {'status': 'success', 'document': document, 'ticker': stock_ticker}

    except Exception as e:
        unsuccessful_tickers.append(stock_ticker)
        return f"ERROR processing {stock_ticker}: {e}"

def parallel_process_stocks(tickers: list, max_workers: int = 10, batch_size: int = 20) -> None:
    processed_count = 0
    total_tickers = len(tickers)

    # Process in batches
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        documents_to_upload = []

        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_ticker = {
                executor.submit(process_stock, ticker): ticker
                for ticker in batch
            }

            for future in concurrent.futures.as_completed(future_to_ticker):
                ticker = future_to_ticker[future]
                try:
                    result = future.result(timeout=30)
                    processed_count += 1

                    if isinstance(result, dict) and result['status'] == 'success':
                        documents_to_upload.append(result['document'])
                        print(f"[{processed_count}/{total_tickers}] Processed {ticker}")
                    else:
                        print(f"[{processed_count}/{total_tickers}] {result}")

                except Exception as exc:
                    print(f'{ticker} generated an exception: {exc}')
                    continue

        # Upload to Pinecone in chunks
        if documents_to_upload:
            chunk_size = 10
            for j in range(0, len(documents_to_upload), chunk_size):
                chunk = documents_to_upload[j:j + chunk_size]

                max_retries = 3
                retry_delay = 5

                for retry in range(max_retries):
                    try:
                        time.sleep(1)  # Minimal delay between uploads
                        vectorstore_from_texts = PineconeVectorStore.from_documents(
                            documents=chunk,
                            embedding=hf_embeddings,
                            index_name=index_name,
                            namespace=namespace
                        )

                        # Track successes
                        for doc in chunk:
                            ticker = doc.metadata.get('Ticker')
                            if ticker:
                                successful_tickers.append(ticker)
                                with open('successful_tickers.txt', 'a') as f:
                                    f.write(f"{ticker}\n")

                        print(f"Successfully uploaded chunk of {len(chunk)} documents to Pinecone")
                        break

                    except Exception as e:
                        print(f"Error uploading to Pinecone (attempt {retry + 1}/{max_retries}): {e}")
                        if retry < max_retries - 1:
                            print(f"Waiting {retry_delay} seconds before retrying...")
                            time.sleep(retry_delay)
                            retry_delay *= 2
                        else:
                            print("Failed to upload chunk after all retries")

# Process remaining tickers
if tickers_to_process:
    print(f"Starting processing of {len(tickers_to_process)} remaining tickers...")
    parallel_process_stocks(
        tickers_to_process,
        max_workers=10,
        batch_size=20
    )
    print("Processing completed!")
else:
    print("No remaining tickers to process!")

Starting processing of 9998 remaining tickers...
[1/9998] Processed MSFT
[2/9998] Processed AVGO
[3/9998] Processed GOOGL
[4/9998] Processed META
[5/9998] Processed TSLA
[6/9998] Processed TSM
[7/9998] Processed NVDA
[8/9998] Processed AAPL
[9/9998] Processed AMZN
[10/9998] Processed BRK-B
[11/9998] Processed LLY
[12/9998] Processed SPY
[13/9998] Processed WMT
[14/9998] Processed UNH
[15/9998] Processed MA
[16/9998] Processed JPM
[17/9998] Processed NVO
[18/9998] Processed XOM
[19/9998] Processed ORCL
[20/9998] Processed V
Successfully uploaded chunk of 10 documents to Pinecone
Successfully uploaded chunk of 10 documents to Pinecone
[21/9998] Processed COST
[22/9998] Processed PG
[23/9998] Processed BAC
[24/9998] Processed JNJ
[25/9998] Processed CRM
[26/9998] Processed HD
[27/9998] Processed NFLX
[28/9998] Processed RCIT
[29/9998] Processed CVX
[30/9998] Processed ABBV
[31/9998] Processed TMUS
[32/9998] Processed KO
[33/9998] Processed ASML
[34/9998] Processed SAP
[35/9998] Processed 

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FACT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FACT&crumb=Xpp9hTXYX7B


[4256/9998] Processed MMLP
[4257/9998] Processed ITRG
[4258/9998] Processed PIM
[4259/9998] Processed MDV
[4260/9998] Skipped FACT - No business summary
Successfully uploaded chunk of 10 documents to Pinecone
Successfully uploaded chunk of 9 documents to Pinecone
[4261/9998] Processed AMLI
[4262/9998] Processed MHF
[4263/9998] Processed WILC
[4264/9998] Processed BFIN
[4265/9998] Processed BMN
[4266/9998] Processed CBNA
[4267/9998] Processed NUTX
[4268/9998] Processed RCKY
[4269/9998] Processed DLNG
[4270/9998] Processed HCVI
[4271/9998] Processed TUSK
[4272/9998] Processed TNYA
[4273/9998] Processed NMG
[4274/9998] Processed MPTI
[4275/9998] Processed EHI
[4276/9998] Processed BFAC
[4277/9998] Processed STHO
[4278/9998] Processed BLNK
[4279/9998] Processed BROG
[4280/9998] Skipped RFAI - No business summary
Successfully uploaded chunk of 10 documents to Pinecone
Successfully uploaded chunk of 9 documents to Pinecone
[4281/9998] Processed MSD
[4282/9998] Processed FNRN
[4283/9998] Proc

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PMVC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PMVC&crumb=Xpp9hTXYX7B


[6754/9998] Processed FECOF
[6755/9998] Processed REOS
[6756/9998] Processed SAML
[6757/9998] Processed XPON
[6758/9998] Skipped PMVC - No business summary
[6759/9998] Processed SVRE
[6760/9998] Processed IMII
Successfully uploaded chunk of 10 documents to Pinecone
Successfully uploaded chunk of 9 documents to Pinecone
[6761/9998] Processed WAST
[6762/9998] Processed CWBR
[6763/9998] Skipped EQOSQ - No business summary
[6764/9998] Processed WORX
[6765/9998] Processed SGBX
[6766/9998] Processed NGTF
[6767/9998] Processed IPSI
[6768/9998] Processed OMQS
[6769/9998] Processed FSTJ
[6770/9998] Processed ILST
[6771/9998] Processed HWKE
[6772/9998] Processed BLMH
[6773/9998] Processed LGHL
[6774/9998] Processed TPTW
[6775/9998] Processed SRSG
[6776/9998] Processed LGCP
[6777/9998] Processed VINO
[6778/9998] Processed MNTR
[6779/9998] Processed HHHEF
[6780/9998] Processed TC
Successfully uploaded chunk of 10 documents to Pinecone
Successfully uploaded chunk of 9 documents to Pinecone
[6781/99

...

[8623/9998] Skipped CHEB-UN - No business summary
[8624/9998] Skipped LUNRW - No business summary
[8625/9998] Processed ATEK-UN
[8626/9998] Skipped CHEB-WT - No business summary
[8627/9998] Skipped IONQ-WT - No business summary
[8628/9998] Processed NMHIW
[8629/9998] Skipped ATEK-WT - No business summary
[8630/9998] Processed CRTDW
[8631/9998] Skipped XFOWW - No business summary
[8632/9998] Skipped DMYY-WT - No business summary
[8633/9998] Skipped AGXRW - No business summary
[8634/9998] Skipped TBMCR - No business summary
[8635/9998] Processed AP-WT
[8636/9998] Processed MDNC
[8637/9998] Processed WTFCP
[8638/9998] Processed CICB
[8639/9998] Processed BURUW
[8640/9998] Processed GLTK
Successfully uploaded chunk of 10 documents to Pinecone
[8641/9998] Skipped VAL-WT - No business summary
[8642/9998] Processed GDEVW
[8643/9998] Processed HSPOR
[8644/9998] Processed HUDAR
[8645/9998] Processed HUDAU
[8646/9998] Processed RCFA-UN
[8647/9998] Processed HSPOW
[8648/9998] Skipped ABLLL - No b

ERROR:yfinance:500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LUCYW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LUCYW&crumb=Xpp9hTXYX7B


[9092/9998] Skipped BRIPF - No business summary
[9093/9998] Skipped BIPJ - No business summary
[9094/9998] Skipped BIPI - No business summary
[9095/9998] Skipped BIPH - No business summary
[9096/9998] Skipped MNQFF - No business summary
[9097/9998] Processed BIP-PB
[9098/9998] Processed BIP-PA
[9099/9998] Processed MNLCF
[9100/9998] Skipped LUCYW - No business summary
Successfully uploaded chunk of 10 documents to Pinecone
Successfully uploaded chunk of 1 documents to Pinecone
[9101/9998] Processed STRRP
[9102/9998] Processed ATH-PD
[9103/9998] Skipped ATHS - No business summary
[9104/9998] Processed CLDT-PA
[9105/9998] Processed HYZNW
[9106/9998] Processed MNUFF
[9107/9998] Processed ATH-PB
[9108/9998] Processed JOCM
[9109/9998] Processed ATH-PC
[9110/9998] Processed ATH-PE
[9111/9998] Processed SAIHW
[9112/9998] Processed ALSAR
[9113/9998] Skipped SIMAW - No business summary
[9114/9998] Skipped BKKT-WT - No business summary
[9115/9998] Processed ALSAU
[9116/9998] Processed ICUCW
[911

ERROR:yfinance:500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PFTAW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PFTAW&crumb=Xpp9hTXYX7B


[9266/9998] Processed PFTAU
[9267/9998] Processed LIFWW
[9268/9998] Processed COF-PI
[9269/9998] Processed LIFWZ
[9270/9998] Processed COF-PN
[9271/9998] Skipped PFTAW - No business summary
[9272/9998] Skipped INPAP - No business summary
[9273/9998] Processed COF-PL
[9274/9998] Processed COF-PK
[9275/9998] Processed DHAIW
[9276/9998] Processed PROCW
[9277/9998] Processed CRESW
[9278/9998] Processed PSPX
[9279/9998] Processed SBEV-WT
[9280/9998] Skipped EPDU - No business summary
Successfully uploaded chunk of 10 documents to Pinecone
Successfully uploaded chunk of 6 documents to Pinecone
[9281/9998] Processed BFRIW
[9282/9998] Processed MYPSW
[9283/9998] Processed BMTX-WT
[9284/9998] Processed TFC-PR
[9285/9998] Processed TFC-PO
[9286/9998] Processed EVVAQ
[9287/9998] Skipped AMBI-WT - No business summary
[9288/9998] Processed DTSTW
[9289/9998] Processed LMMY
[9290/9998] Processed TFC-PI
[9291/9998] Processed ATMP
[9292/9998] Processed COWTF
[9293/9998] Processed DJP
[9294/9998] Proces

In [ ]:
'''
def process_stock(stock_ticker: str) -> str:
    # Skip if already processed
    if stock_ticker in successful_tickers:
        return f"Already processed {stock_ticker}"

    try:
        # Get and store stock data
        stock_data = get_stock_info(stock_ticker)
        stock_description = stock_data['Business Summary']

        # Store stock description in Pinecone
        vectorstore_from_texts = PineconeVectorStore.from_documents(
            documents=[Document(page_content=stock_description, metadata=stock_data)],
            embedding=hf_embeddings,
            index_name=index_name,
            namespace=namespace
        )

        # Track success
        with open('successful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        successful_tickers.append(stock_ticker)

        return f"Processed {stock_ticker} successfully"

    except Exception as e:
        # Track failure
        with open('unsuccessful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        unsuccessful_tickers.append(stock_ticker)

        return f"ERROR processing {stock_ticker}: {e}"

def parallel_process_stocks(tickers: list, max_workers: int = 10) -> None:
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {
            executor.submit(process_stock, ticker): ticker
            for ticker in tickers
        }

        for future in concurrent.futures.as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                result = future.result()
                print(result)

                # Stop on error
                if result.startswith("ERROR"):
                    print(f"Stopping program due to error in {ticker}")
                    executor.shutdown(wait=False)
                    raise SystemExit(1)

            except Exception as exc:
                print(f'{ticker} generated an exception: {exc}')
                print("Stopping program due to exception")
                executor.shutdown(wait=False)
                raise SystemExit(1)

# Prepare your tickers
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

# Process them
parallel_process_stocks(tickers_to_process, max_workers=10)
'''

Already processed BOOT
Already processed FUL
Already processed AHR
Already processed AB
Already processed TXNM
Already processed SIM
Already processed PLXS
Already processed GVA
Already processed HOG
Already processed FTDR
Already processed ROYMY
Already processed GOLF
Already processed SANM
Already processed GBTG
Already processed NXE
Already processed PRIM
Already processed SMG
Already processed ALIT
Already processed OGS
Already processed SIG
Already processed CPA
Already processed PAGP
Already processed HIMS
Already processed MHO
Already processed M
Already processed FRO
Already processed WHD
Already processed GDS
Already processed CORZ
Already processed JOBY
Already processed SBRA
Already processed INTA
Already processed RXO
Already processed LBTYA
Already processed IBOC
Already processed BBIO
Already processed LRN
Already processed GSHD
Already processed FLG
Already processed SKYW
Already processed BHVN
Already processed TEO
Already processed ITGR
Already processed HAE
Already pr

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FACT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FACT&crumb=7HKNS.f1sUA
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PMVC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PMVC&crumb=7HKNS.f1sUA
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LEC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LEC&crumb=7HKNS.f1sUA
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DRDB?modules=financialData%2CquoteType%2CdefaultKeyStatist

SystemExit: 1

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

In [ ]:
client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)

In [ ]:
# Initialize the enhanced RAG model
rag_model = EnhancedRAGModel()

# Example query with filters for finding companies that build data centers
query = "What are companies that build data centers?"
filters = {
    'marketCap': 1000000000,  # Filter for companies > $1B market cap
    'sector': 'Technology'     # Filter for technology sector
}

# Get response with filters
response = rag_model.process_query(
    query=query,
    vector_store=vectorstore,
    client=client,
    filters=filters
)

print("\nEnhanced Stock Analysis:")
print(response)


Enhanced Stock Analysis:
Based on the query, several companies stand out as prominent builders of data centers. Here's a detailed analysis of some of the key players:

**1. Equinix (EQIX)**

Equinix is a leading global data center company that designs, builds, and operates a network of data centers that enable interconnection between companies. They specialize in providing data center infrastructure and interconnection services to a wide range of industries, including cloud and IT providers, enterprises, and network providers.

Key business metrics:

* Market capitalization: approximately $72 billion (as of 2022)
* Revenue growth: 11% CAGR from 2018 to 2022
* Operating margin: 22.6% (2022)
* Total data center portfolio: 237 data centers across 26 countries (2022)

Competitive advantages:

* Interconnection-focused business model, which allows customers to connect with multiple partners and networks within Equinix data centers
* Global reach and extensive network of data centers
* Stro

In [ ]:
#query = "What are some companies that manufacture consumer hardware?"
#raw_query_embedding = get_huggingface_embeddings(query)
#top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)
#top_matches

{'matches': [{'id': '5efda767-c70c-4312-9234-a595f560112c',
              'metadata': {'52 Week High': 2.74,
                           '52 Week Low': 2.14,
                           'Beta': 0.466,
                           'Book Value': 5.91,
                           'Business Summary': 'Deswell Industries, Inc. '
                                               'manufactures and sells '
                                               'injection-molded plastic parts '
                                               'and components, electronic '
                                               'products and subassemblies, '
                                               'and metallic molds and '
                                               'accessory parts for original '
                                               'equipment manufacturers and '
                                               'contract manufacturers in '
                                               'China, the Unite

In [ ]:
#contexts = [item['metadata']['text'] for item in top_matches['matches']]
#augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [ ]:
print(response)

Based on the context provided, some companies that manufacture consumer hardware are:

1. Deswell Industries, Inc. - They manufacture plastic parts and components for consumer products, such as electronic entertainment products, power tools, and outdoor equipment. They also produce electronic products, including audio equipment, consumer audio products, and medical products.

2. Forward Industries, Inc. - They source and distribute carrying cases and other accessories for portable electronic products. They also provide hardware and software product design and engineering services.

3. Deswell Industries, Inc. and Richardson Electronics, Ltd are also associated in producing electronic and audio related components and products where it incorporates elements of consumer electronics however the components can be used across business, industries and end users beyond regular consumers.

4. ROHM Co., Ltd. - They manufacture and sell electronic components, including ICs, discrete semiconductor

# New Section